In [22]:
# Установить необходимые модули
# !pip install pyspark
# !pip install sqlalchemy
#!pip install psycopg2-binary


In [11]:
from sqlalchemy import create_engine,Column,Integer,String,DateTime,ForeignKey # ORM 
from typing import List,Optional
from datetime import datetime
from sqlalchemy.orm import DeclarativeBase # базовый класс для создания декларативных моделей SQLAlchemy
from sqlalchemy.orm import Mapped # типизированный контейнер, используемый для аннотаций типов 
from sqlalchemy.orm import mapped_column # функция используется для определения столбцов в декларативных моделях
from sqlalchemy.orm import relationship # для создания связей между моделями
from sqlalchemy.orm import Session # класс, представляющий сессию ORM
from sqlalchemy.orm import sessionmaker # функция, создающая новый экземпляр Session

import pandas as pd
import numpy as np

In [12]:
brands =pd.read_csv('data/brands.csv')
categories =pd.read_csv('data/categories.csv')
customers =pd.read_csv('data/customers.csv')
order_items =pd.read_csv('data/order_items.csv')
orders =pd.read_csv('data/orders.csv')
products =pd.read_csv('data/products.csv')
staffs =pd.read_csv('data/staffs.csv')
stocks =pd.read_csv('data/stocks.csv')
stores =pd.read_csv('data/stores.csv')

In [16]:
len(customers['email'].unique())

1445

In [18]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  1445 non-null   int64 
 1   first_name   1445 non-null   object
 2   last_name    1445 non-null   object
 3   phone        178 non-null    object
 4   email        1445 non-null   object
 5   street       1445 non-null   object
 6   city         1445 non-null   object
 7   state        1445 non-null   object
 8   zip_code     1445 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 101.7+ KB


In [17]:
customers

,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,NaN,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,NaN,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,NaN,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,NaN,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820
...,...,...,...,...,...,...,...,...,...
1440,1441,Jamaal,Morrison,NaN,jamaal.morrison@msn.com,796 SE. Nut Swamp St.,Staten Island,NY,10301
1441,1442,Cassie,Cline,NaN,cassie.cline@gmail.com,947 Lafayette Drive,Brooklyn,NY,11201
1442,1443,Lezlie,Lamb,NaN,lezlie.lamb@gmail.com,401 Brandywine Street,Central Islip,NY,11722
1443,1444,Ivette,Estes,NaN,ivette.estes@gmail.com,88 N. Canterbury Ave.,Canandaigua,NY,14424


In [10]:
class BaseTable(DeclarativeBase):
        pass

In [ ]:

# Create Models : Table
class Customer(BaseTable):
    __tablename__ = 'customer' # имя таблицы

    id = Column(Integer, primary_key=True)
    first_name = Column(String(80))
    last_name = Column(String(80))
    phone = Column(String(80))
    email = Column(String(120), unique=True)
    street = Column(String(80))
    city = Column(String(80))
    state = Column(String(10))
    city = Column(String(80))
    zip_code = Column(Integer)

    def __repr__(self):
        '''
        Выводы класса в шаблон
        '''
        return f"<User(id={self.id}, email='{self.email}')>"

In [23]:
database_name = 'homework_dbms'
db_user = 'mluser'
db_user_password = 'mlpassword'
server_name = '127.0.0.1'
server_port = '5432'
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_user_password}@{server_name}:{server_port}/{database_name}', echo=True)

'''
sessionmaker() создает функцию для создания сессий ORM.
autocommit=False отключает автоматическое подтверждение транзакций.
autoflush=False отключает автоматическое обновление состояния сессии после каждого запроса.
bind=engine связывает сессию с созданным движком.
'''
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)